# COSMO Project
By Mathilde Raynal, Etienne Bonvin and Xavier Pantet

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from regressions import *

In [3]:
DATA_FOLDER = "data/"
X = np.load(DATA_FOLDER + "feature_mat_radial_compression.npy")
y = np.load(DATA_FOLDER + "CSD500-r_train-H_total.npy")

In [4]:
print("X: " + str(X.shape))
print("y: " + str(y.shape))

X: (30049, 15961)
y: (30049,)


## Idea 1: Good ol' least squares

We first try a standard and naive implementation of `least_squares`:

In [5]:
#w_star = least_squares(y, X)
#loss = rmse(y, X, w_star)
#print("Loss = " + str(loss))

We see that the loss is quite large! We hope to do better using polynomial expansion:

In [21]:
# Sample data to avoid memory errors
N_samples = 1000
N_features = 100
random_row_indices = np.random.randint(0, X.shape[0], N_samples)
random_col_indices = np.random.randint(0, X.shape[1], N_features)
y_sampled = y[random_row_indices]
X_sampled = ((X[random_row_indices].T)[random_col_indices]).T

In [25]:
for degree in range(1, 100):
    X_poly = build_poly(X_sampled, degree)
    w_star = least_squares(y_sampled, X_poly)
    loss = rmse(y, X_poly, w_star)
    print("Degree " + str(degree) + ", loss = " + str(loss))

ValueError: operands could not be broadcast together with shapes (30049,) (1000,) 